# Problem set 1, Asymmetric loss regression

Let's load the data and plot the training / val data.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


# Load data
dataTrain=np.loadtxt('housing-train.txt')
dataVal=np.loadtxt('housing-val.txt')

X_train=(dataTrain[:,np.newaxis,0])
y_train=dataTrain[:,1]
X_val=dataVal[:,np.newaxis,0]
y_val=dataVal[:,1]

plt.plot(X_train, y_train, 'k.')
plt.plot(X_val, y_val, 'b.')


First, we will solve the linear regression (under the "normal" symmetric squared loss) using the closed form solution. Let's define a couple of functions we will need.

In [ ]:
def symmLoss(X, w ,y):
    """
    Get the symmetric squared loss given data X, weight w and ground truth y

    Parameters
    ----------
    X : 2D array
        N x d+1 data matrix (row per example)
    w : 1D array
        d+1 length vector
    y : 1D array
        Observed function values

    Returns
    -------
    loss : a scalar
        The loss calculated by the symmetric loss formula
    """
    
    return   *********** expression for calculating the loss 

Note that the loss we have defined is the normalized squared loss

In [ ]:
def lsqClosedForm(X, y):
    """
    Use closed form solution for least squares minimization

    Parameters
    ----------
    X : 2D array
        N x d+1 data matrix (row per example)
    y : 1D array
        Observed function values

    Returns
    -------
    w : 1D array
        d+1 length vector
    """
    return np.dot(np.linalg.pinv(X), y) # lecture 2

Test the closed form solution: generate a toy data set from a random linear function with <b>no noise</b>. We should be able to perfectly recover w in this case (up to numerical precision).

In [ ]:
X = np.hstack((np.ones([20,1]),np.random.random((20,1))))
w = np.random.random((2))
y = np.dot(X,w)
print('true weight:  '+repr(w))
w_ = lsqClosedForm(X, y)
print('function output: '+repr(w_))
if (np.allclose(w,w_)):
    print('Close enough')
else:
    print('Uh-oh')

The function to estimate the variance of the noise and the log likelihood of the data.

In [ ]:
def logLikelihood(X, w, y):
    """
    Get the estimated variance, and the log likelihood of the data

    Parameters
    ----------
    X : 2D array
        N x d+1 design matrix (row per example)
    w : 1D array
        d+1 length vector
    y : 1D array
        Observed function values

    Returns
    -------
    simga2 : a scalar
        The estimated variance (sigma squared)
    loglike : a scalar
        The log-likelihood under the Gaussian noise model N(0,sigma2)
    """
    N = X.shape[0]   # number of rows in X
    
    # now estimate the variance of the Gaussian noise
    sigma2 = **************** your code for estimating noise variance here
    
    # normalized log-likelihood (mean of per-data point log-likelihood of the model given by w,sigma2)
    loglike = ******************** your code for computing normalized (i.e., average) log-likelihood of the model w on X,y
    
    return sigma2, loglike

Now let's fit linear, quadratic and cubic models to the training data, and plot the fit function. First we need to define the function that will map the original input to an expanded feature vector (including the constant term).

In [ ]:
def degexpand(X, deg, C=None):
    """
    Prepares data matrix with a column of ones and polynomials of specified
    degree.

    Parameters
    ----------
    X : 2D array
        n x d data matrix (row per example)
    deg : integer
        Degree of polynomial
    C : 1D array
        Scaling weights. If not specifed, weights are calculated to fit each
        columns of X in [-1, 1].
        Note: It is shown in problem set 1 that this normalization does
        not affect linear regression, as long as it is applied
        consistently to training *and* test data.

    Returns
    -------
    out_X : 2D array
        n x ((deg*d) + 1) data matrix (row per example)
        *******Fill the comments assuming d = 1*******
        The output is arranged as follows:
            - X[:, 0] is all ones 
            - X[:, 1] is FILL HERE
                .
                .
                .
            - X[:,-2] is FILL HERE
            - X[:,-1] is FILL HERE
    C : 1D array
        Scaling weights that were used. Can be used in scaling other data later in the same way.
    """
    ******** Provide comments where ever asked for (this is the only place where you are asked to fill comments) ****************
    assert X.ndim == 2
    n, m = X.shape

    # Make polynomials
    out_X = (X[..., np.newaxis] ** (1. + np.arange(deg))).reshape(n, -1)

    # Add column of ones
    out_X = np.concatenate([np.ones((out_X.shape[0], 1)), out_X], axis=1)

    if C is None:
        C = abs(out_X).max(axis=0)
    else:
        assert np.shape(C) == (out_X.shape[1],), "C must match outgoing matrix"

    out_X /= C # divide dimension-wise
    return out_X, C


We can now fit models to the training data.

In [ ]:
min_loss = np.Inf
plt.plot(X_train, y_train, 'k.')

# Try different polynomial degrees (feel free to add additional values to explore)
for deg in [1,2,3,5]:
    # Expand data first; you can check how this function works in utils.py
    X, C = degexpand(X_train, deg)
    y = y_train
    
    # Get the result by applying closed-form equation
    w = lsqClosedForm(X, y)
    
    # compute loss on training set
    loss = symmLoss(X, w, y)
    
    # compute loss on val set; note -- use the same scaling matrix C as for training
    val_loss = symmLoss(degexpand(X_val, deg, C)[0], w, y_val)
    
    print('degree %d:' %(deg))
    print('train loss %.6f' %(loss))
    print('val loss %.6f' %(val_loss))
    print('sigma^2: %.6f \nlog-likelihood %.6f\n' %logLikelihood(X, w, y))
    
    ********************** insert code to keep track of the best model so far
    
    # write the code above so that best_param[0] = w of the best model, 
    #                              best_param[1]=deg of the best model, 
    #                              best_param[2]=C (scaling matrix) of the best model
    
    # Plot the function
    color = {1:'b', 2:'g', 3:'r', 5:'c'}[deg]
    plt.plot(np.linspace(min(X_train)-.1,max(X_train)+.1), np.dot(degexpand(np.linspace(min(X_train)-.1,max(X_train)+.1).reshape((50, 1)), deg, C)[0], w), color,label='deg'+str(deg))


                             
    
axes = plt.gca()
axes.set_xlim([0,40])
axes.set_ylim([0,55])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
plt.close()

In [ ]:
# Announce result of best selected model on validation data

dataTest=np.loadtxt('housing-val.txt')
print("Best degree:"+repr(best_param[1])+", loss (sym) on val:")
print(symmLoss(degexpand(X_val, best_param[1], best_param[2])[0], best_param[0], y_val))

(explain your logic for selecting the model)

Now we want to repeat the experiment above but under the asymmetric loss function. Since there is no closed form solution, we will need to rely on gradient descent. First we need to implement the loss function, including the option of calculating the gradient of loss with respect to model parameters

In [ ]:
def asymmLoss(X, w, y,alpha,_grad=False):
    """
    Get the asymmetric loss given data X, weight w and ground truth y

    Parameters
    ----------
    X : 2D array
        N x d+1 design matrix (row per example)
    w : 1D array
        d+1 length vector
    y : 1D array
        Observed function values
    alpha : scalar
        indicating multiplicative factor on underprediction loss (when prediction is lower than ground truth)
    _grad : boolean
        if True, compute and return gradient of the loss w.r.t. w on the data

    Returns
    -------
    loss : a scalar
        The loss calculated by equation in problem set 1
    grad : 1D array  (only when _grad == True)
        vector of same dimension as w, containing the gradient of loss w.r.t. w
    """
       
    ********** enter your code -- you should be able to largely replicate code in symmLoss, 
    ********** with some modifications to account for alpha


    if _grad:
        grad = *************** calculation for gradient
        return loss, grad
    else:
        return loss
    

Advice: It might beneficial to test the loss and gradient function using cooked up X, w, y, alpha

Since we have had the functions to calculate loss and gradient, we can implement the gradient descent algorithm.

In [ ]:
def gradDescent(X, y,alpha,tol=1e-4,maxIt=10000,checkit=1000,verbose=2,lr=0.1):
    """
    Use gradient descent to min(loss(X, w, y))

    Parameters
    ----------
    X : 2D array
        N x d+1 design matrix (row per example)
    y : 1D array
        Observed function values
    alpha : scalar
        weight on positive errors in asymmetric loss
    tol : scalar
        tolerance on loss decrease (default 1e-4)
    maxIt : scalar
        max allowed number of iterations (default 10000)
    checkit : scalar
        # of iterations after which the progress is checked for convergence
    verbose : scalar
        0  -- no progress report; 1 -- every checkit*10; 2 -- every checkit (default 2)
    lr : scalar
        learning rate (default 0.1)

    Returns
    -------
    w : 1D array
        d+1 length vector
        
    it: number of iterations until convergence
    """
    # Random initialize the weight
    w = np.random.randn(X.shape[1])
    it = 0
    lastloss = np.Inf
    while True:
        it += 1 # advance iteration count
        loss, grad = asymmLoss(X, w, y,alpha,_grad=True) # evaluate loss and compute gradient
        
        if it % checkit == 0:
            converged = it >= maxIt or loss > lastloss-tol
            lastloss = loss
            if verbose==2 or verbose==1 and it % (10*checkit) == 0:
                print('iter %d:  loss %.4f' %(it,loss))
            if converged:
                break
            
        w = **************** add code here # update w (only if continuing updates)
       
        
    return w, it

Test gradient descent using any alpha and data generated by (random) noiseless linear model; we should recover the true w accurately (although possibly with less accuracy than the closed form solution for alpha=1)

In [ ]:
X = np.hstack((np.ones([20,1]),np.random.random((20,1))))
w = np.random.random((2))
y = np.dot(X,w)
print('true weight:'+repr(w))
w_, it_ = gradDescent(X, y,10,maxIt=10000,tol=1e-6)
print('%d iterations' %it_)
print('function output:'+repr(w_))

Now we can start testing the preformance of models using aymmetric loss.

In [ ]:
min_loss = np.Inf

plt.plot(X_train, y_train, '.')


for deg in[1,2,3,5]:
    # Expand data first
    X, C = degexpand(X_train, deg)
    y = y_train
        
    # Run gradient descent
    w, _ = gradDescent(X, y,alpha,maxIt=50000,checkit=1000,verbose=1,lr=1)
    loss = asymmLoss(X, w, y,alpha)
    val_loss = asymmLoss(degexpand(X_val, deg, C)[0], w, y_val,alpha)
    
    print('degree %d:' %(deg))
    print('train loss %.6f' %(loss))
    print('val loss %.6f\n' %(val_loss))
    
    ************** code for selecting best model
    # your code should store best_param_asym in the same format as for the symmetric loss experiments above
        
    # Plot the function
    color = {1:'b', 2:'g', 3:'r', 5:'c'}[deg]
    plt.plot(np.linspace(min(X_train)-.1,max(X_train)+.1), np.dot(degexpand(np.linspace(min(X_train)-.1,max(X_train)+.1).reshape((50, 1)), deg, C)[0], w), color,label='deg'+str(deg))
    
axes = plt.gca()
axes.set_xlim([0,40])
axes.set_ylim([0,55])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
plt.close()

In [ ]:
# Announce result on validation data

dataTest=np.loadtxt('housing-val.txt')
print("Best degree:"+repr(best_param[1])+", loss (asym) on val:")
print(asymmLoss(degexpand(X_val, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_val,alpha))

In [ ]:
# Now measure results on test with asymmetric loss, for both models (one learned with sym loss and one learned with asym loss)

dataTest=np.loadtxt('housing-test.txt')
X_test=(dataTest[:,np.newaxis,0])
y_test=dataTest[:,1]

print("Best degree (sym):"+repr(best_param[1]))
alpha = ******* #your alpha when over-estimating is 5 times worse 
print('Sym loss: %.4f' %symmLoss(degexpand(X_test, best_param[1], best_param[2])[0], best_param[0], y_test))
alpha = ******* #your alpha when over-estimating is 20 times worse
print('Asym loss: %.4f' %asymmLoss(degexpand(X_test, best_param[1], best_param[2])[0], best_param[0], y_test,alpha))
print('Log-likelihood (symmetric): %.4f\n' %logLikelihood(degexpand(X_test, best_param[1], best_param[2])[0], best_param[0], y_test)[1])

 

print("Best degree (asym):"+repr(best_param_asym[1]))
print('Sym loss: %.4f' %symmLoss(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test))
alpha = ******* #your alpha when over-estimating is 5 times worse
print('Asym loss: %.4f' %asymmLoss(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test,alpha))
alpha = ******* #your alpha when over-estimating is 20 times worse
print('Asym loss: %.4f' %asymmLoss(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test,alpha))
print('Log-likelihood (symmetric): %.4f\n' %logLikelihood(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test)[1])


'''
print("Best degree (asym):"+repr(best_param_asym[1]))
print('Asym loss: %.4f' %asymmLoss(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test,alpha))
print('Sym loss: %.4f' %symmLoss(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test))
print('Log-likelihood (symmetric): %.4f\n' %logLikelihood(degexpand(X_test, best_param_asym[1], best_param_asym[2])[0], best_param_asym[0], y_test)[1])
'''


We are done with the experiments. Here you should write clearly and succinctly what your conclusions are: what model is better? Why? In what sense? See problem set for more instructions.